In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 50)  # показывать больше строк
pd.set_option('display.max_columns', 50)  # показывать больше колонок

In [38]:
df = pd.read_csv('g:\SkillFactory\EDA\stud_math.csv')

In [7]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
0,GP,F,18,U,NaN,A,4.0,4.0,at_home,teacher,course,mother,2.0,2.0,0.0,yes,no,no,no,yes,-6.0,yes,NaN,no,4.0,3.0,4.0,3.0,6.0,30.0
1,GP,F,17,U,GT3,NaN,1.0,1.0,at_home,other,course,father,1.0,2.0,0.0,no,yes,no,no,no,-6.0,yes,yes,no,5.0,3.0,3.0,3.0,4.0,30.0
2,GP,F,15,U,LE3,T,1.0,1.0,at_home,other,other,mother,1.0,2.0,3.0,yes,no,NaN,no,yes,-6.0,yes,yes,NaN,4.0,3.0,2.0,3.0,10.0,50.0
3,GP,F,15,U,GT3,T,4.0,2.0,health,NaN,home,mother,1.0,3.0,0.0,no,yes,yes,yes,yes,-9.0,yes,yes,yes,3.0,2.0,2.0,5.0,2.0,75.0
4,GP,F,16,U,GT3,T,3.0,3.0,other,other,home,father,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,no,no,4.0,3.0,2.0,5.0,4.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2.0,2.0,services,services,course,NaN,1.0,2.0,2.0,no,yes,yes,no,yes,-6.0,yes,no,no,5.0,5.0,4.0,4.0,212.0,45.0
391,MS,M,17,U,LE3,T,3.0,1.0,services,services,course,mother,2.0,1.0,0.0,no,no,no,no,no,-3.0,yes,yes,no,2.0,4.0,5.0,2.0,3.0,80.0
392,MS,M,21,R,GT3,T,1.0,1.0,other,other,course,NaN,1.0,1.0,3.0,NaN,no,no,no,no,-3.0,NaN,no,no,5.0,5.0,3.0,3.0,3.0,35.0
393,MS,M,18,R,LE3,T,3.0,2.0,services,other,course,mother,3.0,1.0,0.0,no,no,no,NaN,no,-3.0,yes,yes,no,4.0,4.0,1.0,5.0,0.0,50.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   school               395 non-null    object 
 1   sex                  395 non-null    object 
 2   age                  395 non-null    int64  
 3   address              378 non-null    object 
 4   famsize              368 non-null    object 
 5   Pstatus              350 non-null    object 
 6   Medu                 392 non-null    float64
 7   Fedu                 371 non-null    float64
 8   Mjob                 376 non-null    object 
 9   Fjob                 359 non-null    object 
 10  reason               378 non-null    object 
 11  guardian             364 non-null    object 
 12  traveltime           367 non-null    float64
 13  studytime            388 non-null    float64
 14  failures             373 non-null    float64
 15  schoolsup            386 non-null    obj

Вариант 1

In [9]:
# функция вычисления перцентилей и выбросов
def q(s):
    IQR = s.quantile(0.75) - s.quantile(0.25)
    perc25 = s.quantile(0.25)
    perc75 = s.quantile(0.75)
    f = perc25 - 1.5*IQR
    l = perc75 + 1.5*IQR
    print(
        '25-й перцентиль: {},'.format(perc25),
        '75-й перцентиль: {},'.format(perc75),
        "IQR: {}, ".format(IQR),
        "Границы выбросов: [{}, {}].".format(f, l))
    return perc25, perc75, f, l

In [10]:
# функция для вывода уникальных элементов и количества пропусков по всей таблице
def inf(s):
    for col in s.columns:
        print(f'колонка {col}')
        print('value_counts')
        print(df[col].value_counts())
        print('value_counts в процентах')
        print(round(df[col].value_counts(normalize=1) * 100), 2)
        print('пропуски в процентах')
        print(round(((df[col].isna().sum() / len(df[col])) * 100), 1))
        print()

In [11]:
# функция расчета стаистических закономерностей
def get_stat_dif(column):
    cols = df.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'score'], 
                        df.loc[df.loc[:, column] == comb[1], 'score']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [12]:
inf(df)

колонка school
value_counts
GP    349
MS     46
Name: school, dtype: int64
value_counts в процентах
GP    88.0
MS    12.0
Name: school, dtype: float64 2
пропуски в процентах
0.0

колонка sex
value_counts
F    208
M    187
Name: sex, dtype: int64
value_counts в процентах
F    53.0
M    47.0
Name: sex, dtype: float64 2
пропуски в процентах
0.0

колонка age
value_counts
16    104
17     98
18     82
15     82
19     24
20      3
22      1
21      1
Name: age, dtype: int64
value_counts в процентах
16    26.0
17    25.0
18    21.0
15    21.0
19     6.0
20     1.0
22     0.0
21     0.0
Name: age, dtype: float64 2
пропуски в процентах
0.0

колонка address
value_counts
U    295
R     83
Name: address, dtype: int64
value_counts в процентах
U    78.0
R    22.0
Name: address, dtype: float64 2
пропуски в процентах
4.3

колонка famsize
value_counts
GT3    261
LE3    107
Name: famsize, dtype: int64
value_counts в процентах
GT3    71.0
LE3    29.0
Name: famsize, dtype: float64 2
пропуски в процентах


In [13]:
df[df.score == 0]

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
128,GP,M,18,R,NaN,T,2.0,2.0,NaN,other,reputation,mother,1.0,1.0,2.0,no,yes,no,yes,yes,-3.0,yes,yes,no,3.0,3.0,3.0,4.0,0.0,0.0
130,GP,F,15,R,GT3,T,3.0,4.0,services,teacher,course,father,2.0,3.0,2.0,no,yes,no,no,yes,-9.0,yes,yes,yes,4.0,2.0,2.0,5.0,0.0,0.0
131,GP,F,15,U,GT3,NaN,1.0,1.0,at_home,other,course,mother,3.0,1.0,0.0,no,yes,no,yes,no,-3.0,yes,yes,yes,4.0,NaN,3.0,4.0,0.0,0.0
134,GP,M,15,R,GT3,T,3.0,4.0,at_home,teacher,course,mother,NaN,2.0,0.0,no,yes,no,no,yes,-6.0,yes,no,yes,5.0,3.0,3.0,5.0,0.0,0.0
135,GP,F,15,U,GT3,T,4.0,4.0,services,NaN,course,mother,1.0,3.0,0.0,no,yes,no,yes,yes,-9.0,yes,yes,yes,4.0,3.0,3.0,5.0,0.0,0.0
136,GP,M,17,R,GT3,T,3.0,4.0,at_home,other,course,mother,3.0,2.0,0.0,no,no,NaN,no,yes,-6.0,yes,no,no,NaN,4.0,5.0,5.0,0.0,0.0
137,GP,F,16,U,GT3,A,3.0,NaN,other,other,NaN,other,NaN,1.0,2.0,no,yes,no,yes,no,-3.0,yes,yes,yes,4.0,3.0,2.0,5.0,0.0,0.0
140,GP,M,15,U,GT3,T,4.0,3.0,teacher,services,NaN,father,2.0,4.0,0.0,yes,yes,no,NaN,yes,-12.0,yes,yes,no,2.0,2.0,2.0,3.0,0.0,0.0
144,GP,M,17,U,GT3,T,2.0,1.0,other,other,home,NaN,1.0,1.0,3.0,no,yes,no,no,yes,-3.0,yes,yes,no,5.0,4.0,5.0,5.0,0.0,0.0
146,GP,F,15,U,GT3,T,3.0,2.0,health,services,home,father,1.0,2.0,3.0,no,NaN,no,no,yes,-6.0,yes,yes,no,3.0,3.0,2.0,3.0,0.0,0.0


In [14]:
df[df.score == 0].describe()

,age,Medu,Fedu,traveltime,studytime,failures,"studytime, granular",famrel,freetime,goout,health,absences,score
count,37.000000,36.000000,34.000000,32.000000,37.000000,35.000000,37.000000,34.000000,36.000000,36.000000,35.000000,36.0,37.0
mean,17.054054,2.333333,2.323529,1.500000,1.945946,0.857143,-5.837838,3.823529,3.111111,3.166667,3.657143,0.0,0.0
std,1.311190,1.041976,1.147344,0.718421,0.911225,1.033067,2.733674,1.028992,0.887300,1.341641,1.304807,0.0,0.0
min,15.000000,1.000000,1.000000,1.000000,1.000000,0.000000,-12.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0
25%,16.000000,1.750000,1.000000,1.000000,1.000000,0.000000,-6.000000,3.000000,3.000000,2.000000,3.000000,0.0,0.0
50%,17.000000,2.000000,2.000000,1.000000,2.000000,1.000000,-6.000000,4.000000,3.000000,3.000000,4.000000,0.0,0.0
75%,18.000000,3.000000,3.000000,2.000000,2.000000,1.000000,-3.000000,4.750000,4.000000,4.000000,5.000000,0.0,0.0
max,19.000000,4.000000,4.000000,3.000000,4.000000,3.000000,-3.000000,5.000000,5.000000,5.000000,5.000000,0.0,0.0


у учеников с нулем баллов неплохие остальные данные, а главное - они не имеют пропусков занятий эти ученики либо не явились на экзамен по разным причинам (скорее всего по уважительной причине), либо эти данные не верны и не могут быть использованы в модели их придется удалить вместе с немногочисленными пропусками

In [39]:
# удаляем 0 баллов за экзамен и пропуски в колонке score
df = df[(df['score'] != 0) & (df['score'].notna())]

In [40]:
# удаляем выбросы 385 и 212 из колонки absences
df = df[(df['absences'] != 385) & (df['absences'] != 212)]

In [41]:
# исправляем неверное значение -1 на 1
df[df.famrel == -1] = 1

In [42]:
# исправляем значение 40 на 4
df[df.Fedu == 40] = 4

In [43]:
# удаляем выбросы
for i in ['age', 'absences', 'score']:
    print(i, len(df))
    if (type(df.loc[0, i]) == type(df.score[0])):
        p25, p75, f, l = q(df[i])
        df = df[(df[i] > f) & (df[i] < l)]

age 350
absences 350
25-й перцентиль: 2.0, 75-й перцентиль: 8.0, IQR: 6.0,  Границы выбросов: [-7.0, 17.0].
score 314
25-й перцентиль: 50.0, 75-й перцентиль: 70.0, IQR: 20.0,  Границы выбросов: [20.0, 100.0].


In [44]:
# заменияем пропуски модой
for i in df.columns:
    df[i] = df[i].fillna(df[i].value_counts().index[0])

In [22]:
inf(df)

колонка school
value_counts
GP    273
MS     38
Name: school, dtype: int64
value_counts в процентах
GP    88.0
MS    12.0
Name: school, dtype: float64 2
пропуски в процентах
0.0

колонка sex
value_counts
F    159
M    152
Name: sex, dtype: int64
value_counts в процентах
F    51.0
M    49.0
Name: sex, dtype: float64 2
пропуски в процентах
0.0

колонка age
value_counts
16    85
17    76
15    72
18    64
19    10
20     2
22     1
21     1
Name: age, dtype: int64
value_counts в процентах
16    27.0
17    24.0
15    23.0
18    21.0
19     3.0
20     1.0
22     0.0
21     0.0
Name: age, dtype: float64 2
пропуски в процентах
0.0

колонка address
value_counts
U    246
R     65
Name: address, dtype: int64
value_counts в процентах
U    79.0
R    21.0
Name: address, dtype: float64 2
пропуски в процентах
0.0

колонка famsize
value_counts
GT3    222
LE3     89
Name: famsize, dtype: int64
value_counts в процентах
GT3    71.0
LE3    29.0
Name: famsize, dtype: float64 2
пропуски в процентах
0.0

кол

In [45]:
df.corr()

,age,Medu,Fedu,traveltime,studytime,failures,"studytime, granular",famrel,freetime,goout,health,absences,score
age,1.000000,-0.141103,-0.128566,0.099415,0.013312,0.213574,-0.013312,0.063161,0.014521,0.138676,-0.027352,0.197038,-0.138685
Medu,-0.141103,1.000000,0.577145,-0.171163,0.071038,-0.210470,-0.071038,-0.015470,0.027662,0.037986,-0.064615,0.096044,0.169272
Fedu,-0.128566,0.577145,1.000000,-0.138991,-0.035228,-0.290585,0.035228,-0.023173,0.002836,0.000894,-0.033510,-0.007587,0.115087
traveltime,0.099415,-0.171163,-0.138991,1.000000,-0.106032,0.102987,0.106032,-0.040049,0.023592,0.038340,-0.003904,-0.040209,-0.081526
studytime,0.013312,0.071038,-0.035228,-0.106032,1.000000,-0.144187,-1.000000,0.037520,-0.130101,-0.009898,-0.073289,-0.045653,0.121042
failures,0.213574,-0.210470,-0.290585,0.102987,-0.144187,1.000000,0.144187,0.023493,0.132187,0.106977,0.072121,0.168896,-0.280758
"studytime, granular",-0.013312,-0.071038,0.035228,0.106032,-1.000000,0.144187,1.000000,-0.037520,0.130101,0.009898,0.073289,0.045653,-0.121042
famrel,0.063161,-0.015470,-0.023173,-0.040049,0.037520,0.023493,-0.037520,1.000000,0.161786,0.037214,0.125235,-0.084475,0.060484
freetime,0.014521,0.027662,0.002836,0.023592,-0.130101,0.132187,0.130101,0.161786,1.000000,0.283618,0.061394,0.077469,-0.004850
goout,0.138676,0.037986,0.000894,0.038340,-0.009898,0.106977,0.009898,0.037214,0.283618,1.000000,0.025964,0.170730,-0.195735


все числовые переменные слабо коррелированные
колонки studytime и studytime, granular полностью скоррелированы, studytime, granular можно удалить из модели

In [24]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 
           'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']:
    get_stat_dif(col)

Найдены статистически значимые различия для колонки sex
Найдены статистически значимые различия для колонки address
Найдены статистически значимые различия для колонки schoolsup


In [51]:
df_for_model = df[['age', 'failures', 'absences', 'sex', 'address', 'schoolsup', 'Medu', 'studytime', 'goout', 'Fedu',
                  'famrel', 'freetime', 'health', 'traveltime']]

In [52]:
df_for_model

,age,failures,absences,sex,address,schoolsup,Medu,studytime,goout,Fedu,famrel,freetime,health,traveltime
0,18,0.0,6.0,F,U,yes,4.0,2.0,4.0,4.0,4.0,3.0,3.0,2.0
1,17,0.0,4.0,F,U,no,1.0,2.0,3.0,1.0,5.0,3.0,3.0,1.0
2,15,3.0,10.0,F,U,yes,1.0,2.0,2.0,1.0,4.0,3.0,3.0,1.0
3,15,0.0,2.0,F,U,no,4.0,3.0,2.0,2.0,3.0,2.0,5.0,1.0
4,16,0.0,4.0,F,U,no,3.0,2.0,2.0,3.0,4.0,3.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,18,0.0,0.0,F,U,no,3.0,2.0,4.0,1.0,4.0,3.0,1.0,1.0
391,17,0.0,3.0,M,U,no,3.0,1.0,5.0,1.0,2.0,4.0,2.0,2.0
392,21,3.0,3.0,M,R,no,1.0,1.0,3.0,1.0,5.0,5.0,3.0,1.0
393,18,0.0,0.0,M,R,no,3.0,1.0,1.0,2.0,4.0,4.0,5.0,3.0


Вариант 2

In [25]:
df = pd.read_csv('g:\SkillFactory\EDA\stud_math.csv')

In [26]:
# удаляем 0 баллов за экзамен и пропуски в колонке score
df = df[(df['score'] != 0) & (df['score'].notna())]

In [27]:
# удаляем выбросы 385 и 212 из колонки absences
df = df[(df['absences'] != 385) & (df['absences'] != 212)]

In [28]:
# исправляем неверное значение -1 на 1
df[df.famrel == -1] = 1

In [29]:
df[df.Fedu == 40] = 4

In [31]:
# удаляем выбросы
for i in ['age', 'absences', 'score']:
    print(i, len(df))
    if (type(df.loc[0, i]) == type(df.score[0])):
        p25, p75, f, l = q(df[i])
        df = df[(df[i] > f) & (df[i] < l)]

age 350
absences 350
25-й перцентиль: 2.0, 75-й перцентиль: 8.0, IQR: 6.0,  Границы выбросов: [-7.0, 17.0].
score 314
25-й перцентиль: 50.0, 75-й перцентиль: 70.0, IQR: 20.0,  Границы выбросов: [20.0, 100.0].


In [33]:
# преобразуем категориальные столбцы, выраженные числами, в строки
for i in ['Medu', 'Fedu', 'traveltime', 'studytime', 'famrel', 'freetime', 'goout', 'health']:
    df[i] = df[i].astype(str)

In [34]:
# заменияем пропуски модой в категориальных столбцах
for i in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 
           'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic',
          'Medu', 'Fedu', 'traveltime', 'studytime', 'famrel', 'freetime', 'goout', 'health']:
    df[i] = df[i].fillna(df[i].value_counts().index[0])

In [35]:
# заменяем пропуски в числовых столбцах медианой
for i in ['age', 'absences', 'score']:
    df[i] = df[i].fillna(df[i].median())

In [36]:
df.corr()

,age,failures,"studytime, granular",absences,score
age,1.000000,0.208326,-0.013460,0.197038,-0.138685
failures,0.208326,1.000000,0.151397,0.167755,-0.282308
"studytime, granular",-0.013460,0.151397,1.000000,0.045640,-0.122529
absences,0.197038,0.167755,0.045640,1.000000,-0.223637
score,-0.138685,-0.282308,-0.122529,-0.223637,1.000000


все переменные слабо скоррелированы
studytime, granular является клоном studytime и может быть удалена из модели

In [37]:
for col in ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 
           'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic',
          'Medu', 'Fedu', 'traveltime', 'studytime', 'famrel', 'freetime', 'goout', 'health']:
    get_stat_dif(col)

Найдены статистически значимые различия для колонки sex
Найдены статистически значимые различия для колонки address
Найдены статистически значимые различия для колонки schoolsup
Найдены статистически значимые различия для колонки Medu
Найдены статистически значимые различия для колонки studytime
Найдены статистически значимые различия для колонки goout


In [47]:
df_for_model = df[['age', 'failures', 'absences', 'sex', 'address', 'schoolsup', 'Medu', 'studytime', 'goout']]

In [48]:
df_for_model

,age,failures,absences,sex,address,schoolsup,Medu,studytime,goout
0,18,0.0,6.0,F,U,yes,4.0,2.0,4.0
1,17,0.0,4.0,F,U,no,1.0,2.0,3.0
2,15,3.0,10.0,F,U,yes,1.0,2.0,2.0
3,15,0.0,2.0,F,U,no,4.0,3.0,2.0
4,16,0.0,4.0,F,U,no,3.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
388,18,0.0,0.0,F,U,no,3.0,2.0,4.0
391,17,0.0,3.0,M,U,no,3.0,1.0,5.0
392,21,3.0,3.0,M,R,no,1.0,1.0,3.0
393,18,0.0,0.0,M,R,no,3.0,1.0,1.0
